In [5]:
import pandas as pd
import numpy as np

#tracking mlflow
import mlflow
import mlflow.sklearn
from time import time
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,hamming_loss, jaccard_score, confusion_matrix, roc_curve, auc)
import matplotlib.pyplot as plt


#Prétraitement
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

nltk.download('stopwords')
nltk.download('wordnet')

#Extraction de features
#Bag of words
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#Word2Vec
#from gensim.models import Word2Vec
#BERT
#from transformers import BertTokenizer, BertModel
#import torch
#USE
#import tensorflow_hub as hub

from sklearn.linear_model import LogisticRegression

#empecher les messages d'erreur
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Formatage taille cellule pour s'adapter aux graphiques
from IPython.display import display, HTML
display(HTML("<style>.output_scroll { height: auto !important; }</style>"))


df = pd.read_csv("./filtered_questions.csv",sep=',', encoding='utf-8')  
df.head(5)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maudt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maudt\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,title,body,link,view_count,score,tags
0,Why is processing a sorted array faster than p...,"<p>In this C++ code, sorting the data (<em>bef...",https://stackoverflow.com/questions/11227809/w...,1894410,27300,"java,c++,performance,cpu-architecture,branch-p..."
1,How do I delete a Git branch locally and remot...,<p>Failed Attempts to Delete a Remote Branch:<...,https://stackoverflow.com/questions/2003505/ho...,11766932,20374,"git,version-control,git-branch,git-push,git-re..."
2,What is the &#39;--&gt;&#39; operator in C/C++?,"<p>After reading <a href=""http://groups.google...",https://stackoverflow.com/questions/1642028/wh...,1018216,10170,"c++,c,operators,code-formatting,standards-comp..."
3,How do I force &quot;git pull&quot; to overwri...,<p>How do I force an overwrite of local files ...,https://stackoverflow.com/questions/1125968/ho...,8726770,9730,"git,version-control,overwrite,git-pull,git-fetch"
4,What and where are the stack and heap?,<ul>\n<li>What are the stack and heap?</li>\n<...,https://stackoverflow.com/questions/79923/what...,1950792,9450,"data-structures,memory-management,heap-memory,..."


## Tracking MLFLOW

Elements du tracking :

1. Métriques de Performance (Scores) :

Accuracy : Précision moyenne des tags prédits.

Precision (micro, macro, weighted) : Précision des prédictions, évaluée de plusieurs manières.

Recall (micro, macro, weighted) : Taux de rappel des prédictions.

F1 Score (micro, macro, weighted) : Moyenne harmonique de la précision et du rappel.

Hamming Loss : Nombre moyen d'étiquettes incorrectes par instance.

Subset Accuracy : Proportion d'instances pour lesquelles toutes les étiquettes prédictes sont correctes.

Jaccard Similarity : Mesure de la similarité entre les ensembles de tags prédits et réels.

2. Hyperparamètres

Paramètres du Modèle :
Pour un Random Forest : n_estimators, max_depth, min_samples_split, min_samples_leaf, etc.
Pour un SVM : C, kernel, gamma, etc.
Pour un modèle de deep learning : learning_rate, batch_size, num_epochs, dropout_rate, etc.

Prétraitement : Taille du vocabulaire.
Méthodes de vectorisation (TF-IDF, embeddings).
Paramètres de tokenisation (n-grams, stop words).

3. Artifacts

Modèles Entraînés : Modèles sauvegardés pour chaque run.

Vecteurs de Features : Vecteurs résultant de la vectorisation des questions.

Fichiers de Logs : Logs de l'entraînement et de la validation.

Matrice de Confusion : Pour visualiser les prédictions correctes et incorrectes.

Courbes ROC/AUC : Pour les modèles supportant predict_proba.

Courbes d'apprentissage : Visualisation de la progression des métriques de performance pendant l'entraînement.

4. Graphiques Intéressants à Visualiser

Courbes d'Apprentissage : Visualiser l'évolution de la perte et des métriques (précision, rappel, F1) pendant l'entraînement.

Matrice de Confusion : Affichage de la matrice de confusion pour comprendre les erreurs de classification.

Courbe ROC : Affichage de la courbe ROC pour les modèles de classification probabilistes.

Importance des Features : Graphique montrant l'importance des différentes features pour les modèles comme les Random Forests.

Distribution des Tags Prédits vs Réels : Comparaison de la distribution des tags prédits par le modèle par rapport aux tags réels.

Histogrammes des Temps de Traitement : Visualisation des temps de traitement pour l'entraînement et la prédiction.

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in np.ndindex(cm.shape):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

def track_classification_experiment(model, X_train, y_train, X_test, y_test, params, class_labels=None):
    with mlflow.start_run():
        # Enregistrement des hyperparamètres
        mlflow.log_params(params)

        # Entraînement du modèle
        start_time = time()
        model.fit(X_train, y_train)
        training_time = time() - start_time
        mlflow.log_metric("training_time", training_time)

        # Prédictions et calcul des scores
        start_time = time()
        predictions = model.predict(X_test)
        prediction_time = time() - start_time
        mlflow.log_metric("prediction_time", prediction_time)
        
        # Calcul des scores
        acc = accuracy_score(y_test, predictions)
        prec_micro = precision_score(y_test, predictions, average='micro')
        rec_micro = recall_score(y_test, predictions, average='micro')
        f1_micro = f1_score(y_test, predictions, average='micro')
        ham_loss = hamming_loss(y_test, predictions)
        subset_acc = np.mean([set(pred) == set(real) for pred, real in zip(predictions, y_test)])
        jaccard = jaccard_score(y_test, predictions, average='samples')

        # Enregistrement des scores
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("precision_micro", prec_micro)
        mlflow.log_metric("recall_micro", rec_micro)
        mlflow.log_metric("f1_score_micro", f1_micro)
        mlflow.log_metric("hamming_loss", ham_loss)
        mlflow.log_metric("subset_accuracy", subset_acc)
        mlflow.log_metric("jaccard_similarity", jaccard)

        # Enregistrement de la matrice de confusion
        if class_labels is not None:
            cm = confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))
            plt.figure()
            plot_confusion_matrix(cm, classes=class_labels, title='Confusion matrix')
            plt.savefig('confusion_matrix.png')
            mlflow.log_artifact('confusion_matrix.png')

        # Enregistrement des courbes ROC (exemple pour les modèles supportant predict_proba)
        if hasattr(model, 'predict_proba'):
            probas = model.predict_proba(X_test)
            for i, class_label in enumerate(class_labels):
                fpr, tpr, thresholds = roc_curve(y_test[:, i], probas[:, i])
                roc_auc = auc(fpr, tpr)
                plt.figure()
                plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlim([0.0, 1.0])
                plt.ylim([0.0, 1.05])
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title(f'ROC Curve for {class_label}')
                plt.legend(loc="lower right")
                plt.savefig(f'roc_curve_{class_label}.png')
                mlflow.log_artifact(f'roc_curve_{class_label}.png')

        # Enregistrement du modèle
        mlflow.sklearn.log_model(model, "model")

        # Enregistrement de l'environnement
        mlflow.set_tag("mlflow_version", mlflow.__version__)
        mlflow.set_tag("model_type", type(model).__name__)
        # Log des versions des packages utilisés (sklearn, numpy, etc.)

### Exemple d'utilisation des fonctions de tracking

In [ ]:
# # Exemple d'utilisation
# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import MultiLabelBinarizer

# # Exemple d'initialisation des données
# X_train = ...  # Vos données d'entraînement
# X_test = ...   # Vos données de test
# y_train = ...  # Vos étiquettes d'entraînement
# y_test = ...   # Vos étiquettes de test

# model = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, max_depth=10))
# params = {"n_estimators": 100, "max_depth": 10}
# mlb = MultiLabelBinarizer()
# y_train_bin = mlb.fit_transform(y_train)
# y_test_bin = mlb.transform(y_test)

# class_labels = mlb.classes_
# track_classification_experiment(model, X_train, y_train_bin, X_test, y_test_bin, params, class_labels)

## Prétraitement de texte

In [57]:
# Fonction pour nettoyer le texte
def clean_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()  # Supprimer les balises HTML
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # Supprimer la ponctuation
    words = text.lower().split()  # Mettre en minuscule et diviser en mots
    stop_words = set(stopwords.words('english'))  # Obtenir les stopwords anglais
    words = [w for w in words if w not in stop_words]  # Supprimer les stopwords
    lemmatizer = WordNetLemmatizer()  # Initialiser le lemmatizer
    words = [lemmatizer.lemmatize(w) for w in words]  # Lemmatization
    return ' '.join(words)

# Appliquer le nettoyage aux colonnes 'title' et 'body'
df['clean_title'] = df['title'].apply(clean_text)
df['clean_body'] = df['body'].apply(clean_text)

# Séparer les tags en liste
df['tags'] = df['tags'].apply(lambda x: x.split(','))

# Binariser les tags
mlb = MultiLabelBinarizer()
df_tags = mlb.fit_transform(df['tags'])

# Séparer les données en jeux d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df[['clean_title', 'clean_body']], df_tags, test_size=0.2, random_state=42)

# Afficher un aperçu des données nettoyées
df['clean_title'].head()


0    processing sorted array faster processing unso...
1                   delete git branch locally remotely
2                                         operator c c
3                  force git pull overwrite local file
4                                           stack heap
Name: clean_title, dtype: object

In [59]:
df['clean_body'].head()

0    c code sorting data timed region make primary ...
1    failed attempt delete remote branch git branch...
2    reading hidden feature dark corner c stl comp ...
3    force overwrite local file git pull local repo...
4    stack heap located physically computer memory ...
Name: clean_body, dtype: object

## Extraction de Features

### Bag of Words

In [61]:
# # Créer le CountVectorizer pour 'title'
# count_vect = CountVectorizer()
# count_vect.fit(X_train['clean_title'])

# # Transformer 'title' et 'body' des jeux d'entraînement et de test
# X_train_count = count_vect.transform(X_train['clean_title'] + " " + X_train['clean_body'])
# X_test_count = count_vect.transform(X_test['clean_title'] + " " + X_test['clean_body'])

# # Créer le TfidfVectorizer pour 'title'
# tfidf_vect = TfidfVectorizer()
# tfidf_vect.fit(X_train['clean_title'])

# # Transformer 'title' et 'body' des jeux d'entraînement et de test
# X_train_tfidf = tfidf_vect.transform(X_train['clean_title'] + " " + X_train['clean_body'])
# X_test_tfidf = tfidf_vect.transform(X_test['clean_title'] + " " + X_test['clean_body'])

### Définir les fonctions de vectorisation

In [ ]:
def vectorize_count(X_train, X_test):
    count_vect = CountVectorizer()
    count_vect.fit(X_train['clean_title'] + " " + X_train['clean_body'])
    X_train_count = count_vect.transform(X_train['clean_title'] + " " + X_train['clean_body'])
    X_test_count = count_vect.transform(X_test['clean_title'] + " " + X_test['clean_body'])
    return X_train_count, X_test_count

def vectorize_tfidf(X_train, X_test):
    tfidf_vect = TfidfVectorizer()
    tfidf_vect.fit(X_train['clean_title'] + " " + X_train['clean_body'])
    X_train_tfidf = tfidf_vect.transform(X_train['clean_title'] + " " + X_train['clean_body'])
    X_test_tfidf = tfidf_vect.transform(X_test['clean_title'] + " " + X_test['clean_body'])
    return X_train_tfidf, X_test_tfidf

### Définir la fonction pour exécuter l'expérience

In [ ]:
def run_experiment(vectorization_method, model, X_train, y_train, X_test, y_test, params, method_name):
    X_train_vect, X_test_vect = vectorization_method(X_train, X_test)
    params['vectorization'] = method_name
    
    track_classification_experiment(
        model=model,
        X_train=X_train_vect,
        y_train=y_train,
        X_test=X_test_vect,
        y_test=y_test,
        params=params,
        class_labels=['clean_title', 'clean_body']  # Remplacez par les labels de votre jeu de données
    )

### Executer les 2 experiences

In [ ]:
# Définir les paramètres du modèle
params = {
    "C": 1.0,
    "penalty": "l2",
    "solver": "liblinear"
}

# Définir le modèle à utiliser (par exemple, la régression logistique)
model = LogisticRegression(C=params["C"], penalty=params["penalty"], solver=params["solver"])

# Exécuter l'expérience pour CountVectorizer
run_experiment(vectorization_method=vectorize_count, model=model, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, params=params, method_name="CountVectorizer")

# Exécuter l'expérience pour TfidfVectorizer
run_experiment(vectorization_method=vectorize_tfidf, model=model, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, params=params, method_name="TfidfVectorizer")

### Word2Vec

In [63]:
# # Préparer les phrases pour Word2Vec
# sentences = [text.split() for text in X_train['clean_title'] + " " + X_train['clean_body']]
# word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# # Fonction pour obtenir la moyenne des embeddings Word2Vec pour un texte
# def get_w2v_features(text, model):
#     words = text.split()
#     word_vecs = [model.wv[word] for word in words if word in model.wv]
#     return np.mean(word_vecs, axis=0) if word_vecs else np.zeros(model.vector_size)

# # Appliquer aux jeux d'entraînement et de test
# X_train_w2v = np.array([get_w2v_features(text, word2vec_model) for text in X_train['clean_title'] + " " + X_train['clean_body']])
# X_test_w2v = np.array([get_w2v_features(text, word2vec_model) for text in X_test['clean_title'] + " " + X_test['clean_body']])

### BERT

In [ ]:
# # Initialiser le tokenizer et le modèle BERT
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = BertModel.from_pretrained('bert-base-uncased')

# # Fonction pour obtenir les embeddings BERT
# def get_bert_features(text, tokenizer, model):
#     inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     return outputs.last_hidden_state.mean(dim=1).numpy()

# # Appliquer aux jeux d'entraînement et de test
# X_train_bert = np.array([get_bert_features(text, tokenizer, bert_model).flatten() for text in X_train['clean_title'] + " " + X_train['clean_body']])
# X_test_bert = np.array([get_bert_features(text, tokenizer, bert_model).flatten() for text in X_test['clean_title'] + " " + X_test['clean_body']])

### USE

In [65]:
# # Charger le modèle USE
# use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# # Fonction pour obtenir les embeddings USE
# def get_use_features(text, model):
#     return model([text]).numpy().flatten()

# # Appliquer aux jeux d'entraînement et de test
# X_train_use = np.array([get_use_features(text, use_model) for text in X_train['clean_title'] + " " + X_train['clean_body']])
# X_test_use = np.array([get_use_features(text, use_model) for text in X_test['clean_title'] + " " + X_test['clean_body']])